In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import os
import sys

In [17]:
with open('/content/drive/MyDrive/news-crawl-corpus/train.news', 'r') as f:
    news_train = f.readlines()
with open('/content/drive/MyDrive/news-crawl-corpus/dev.news', 'r') as f:
    news_validation = f.readlines()
with open('/content/drive/MyDrive/news-crawl-corpus/test.news', 'r') as f:
    news_test = f.readlines()


In [18]:
#print length of corpus
print('train:', len(news_train))  
print('valid:', len(news_validation))

train: 20000
valid: 500


In [19]:
import re
def preprocess(text):
    #the function prpocess a single line
    #it takes the single line and returns a list of tokens for that particular line
    #make text lower
    cleaned_text = text.lower()
    # cleaned_text = text
    #remove non-ASCII characters
    #cleaned_text = re.sub(r'[^\x00-\x7F]+',' ', cleaned_text)
    # remove URLS
    cleaned_text = re.sub(r"http\S+", "<URL>", cleaned_text)
    # remove HTs
    cleaned_text = re.sub(r"#[A-Za-z0-9_]+", "<HASHTAG>", cleaned_text)
    # remove Mentions
    cleaned_text = re.sub(r"@[A-Za-z0-9_]+", "<MENTION>", cleaned_text)
    #replace percentage quantities with tags
    cleaned_text = re.sub(r'(\d+(\.\d+)?%)',"<PERCENT>",cleaned_text)
    #replace numbers with tags
    cleaned_text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " <NUM> ", cleaned_text)
    #hypenated words are accounted for by joining them/merging them together
    cleaned_text = re.sub(r'\w+(?:-\w+)+', '', cleaned_text)
    # Substitue for punctuations
    # cleaned_text = re.sub(r"(\'t)"," not",cleaned_text)
    # cleaned_text = re.sub(r'(i\'m)',"i am",cleaned_text)
    # cleaned_text = re.sub(r'(ain\'t)',"am not",cleaned_text)
    # cleaned_text = re.sub(r'(\'ll)'," will",cleaned_text)
    # cleaned_text = re.sub(r'(\'ve)'," have",cleaned_text)
    # cleaned_text = re.sub(r'(\'re)'," are",cleaned_text)
    # cleaned_text = re.sub(r'(\'s)'," is",cleaned_text)
    # cleaned_text = re.sub(r'(\'re)'," are",cleaned_text)
    #removing repetetive spam
    cleaned_text = re.sub('\!\!+', '!', cleaned_text)
    cleaned_text = re.sub('\*\*+', '*', cleaned_text)
    cleaned_text = re.sub('\>\>+', '>', cleaned_text)
    cleaned_text = re.sub('\<\<+', '<', cleaned_text)
    cleaned_text = re.sub('\?\?+', '?', cleaned_text)
    cleaned_text = re.sub('\!\!+', '!', cleaned_text)
    cleaned_text = re.sub('\.\.+', '.', cleaned_text)
    cleaned_text = re.sub('\,\,+', ',', cleaned_text)
    #matching punctuation characters at end of sentences and padding them
    cleaned_text = re.sub('([;:.,!?()])', r' \1 ', cleaned_text)
    #removing multiple spaces finally
    cleaned_text = re.sub('\s{2,}', ' ', cleaned_text)
    #remove trailing white spaces
    cleaned_text = re.sub(r'\s+$', '', cleaned_text) #important to get rid of empty tokens at the end of list
    #tokenization based on spaces for each line
    # spaces = r"\s+"
    # tokenized_sent = re.split(spaces, cleaned_text)
    return cleaned_text

In [20]:
#preprocess the corpus
news_train_clean = [preprocess(line) for line in news_train]
news_validation_clean = [preprocess(line) for line in news_validation]
news_validation_clean[:10]

["dans une vidéo publiée vendredi matin , le président obama affirme que la a été inondée de messages d'appui pour renforcer les lois sur les armes à feu au cours de la dernière semaine .",
 "une fois la canne et le moulinet bien en mains , il n'est pas rare de faire de belles prises .",
 'court 1 : a partir de 14h00 ( 9h30 h . f . )',
 'comme sur les situations de coups de pied arrêtés .',
 "monahan domine de son côté les pointeurs des 67's avec <NUM> points en <NUM> matches .",
 'chaque membre touchera <NUM> $ en boni de signature .',
 "en <NUM> matchs , il a marqué <NUM> buts et ajouté <NUM> mentions d'aide pour <NUM> points .",
 "aujourd'hui , numéro 4 : ian poulter .",
 "cette année , hanouka s'est amorcée le <NUM> décembre .",
 'un anniversaire et un enterrement']

In [21]:
news_test_clean = [preprocess(line) for line in news_test]

In [22]:
def create_UNK_corpus(data):
    freq_dict = {}
    for line in data:
        for word in line.split():
            if word in freq_dict:
                freq_dict[word] += 1
            else:
                freq_dict[word] = 1
    freq_dict['<UNK>'] = 0
    #replace all words with frequency less than 3 with <UNK> in freq_dict
    word_list = []
    for word in freq_dict:
        if freq_dict[word] == 1:
            freq_dict["<UNK>"] += freq_dict[word]
            #remove the word from the freq_dict
            word_list.append(word)
    for word in word_list:
        del freq_dict[word]
    #for each sentence in corpus if word not in freq_dict replace with <UNK>
    temp = []
    for line in data:
        for word in line.split():
            if word not in freq_dict:
                line = line.replace(word, "<UNK>")
        temp.append(line)
    return temp
news_train_clean = create_UNK_corpus(news_train_clean)
news_train_clean[:10]


["au pied des pyramides de <UNK> , l'air est exactement le même .",
 "l'un des effets <UNK> du cancer est la détresse psychologique .",
 "les thrashers , qui avaient le deuxième choix , ont donc accepté d'échanger ce choix aux canucks en retour du tout premier .",
 "selon lui , le président des , barack obama et la présidente de la commission d'enquête sur l'octroi et la gestion des contrats publics dans l'industrie de la construction , la juge france charbonneau , sont deux des trois personnalités qui ont occupé l'actualité , aux côtés de la première ministre du québec , pauline marois .",
 'une seule de ses plumes tombée à terre , avait , , suffi à <UNK> le monde . mais comment trouver sa majesté ?',
 "en plus de cette rémunération de très nombreux avantages viendront s'y ajouter ( participation et intéressement , ce , . )",
 "depuis cet été , cependant , de belles choses semblent poindre à l'horizon pour le jeune homme de <NUM> ans .",
 '"notre but à tous les deux , c\'est de faire 

In [23]:
import torch
import os
import numpy as np

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()
    def get_data(self, train_data, batch_size):
        tokens = 0
        for line in train_data:
            #tokenize the line
            words = ['<START>'] + line.split() + ['<END>']
            tokens += len(words)
            for word in words:
                self.dictionary.add_word(word)

        #tokenizing the content
        ids = torch.LongTensor(tokens) #contains the ids of the words
        token = 0
        for line in train_data:
            words = ['<START>'] + line.split() + ['<END>']
            for word in words:
                ids[token] = self.dictionary.word2idx[word]
                token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        ##return reshaped tensor using .view()
        ## the -1-> situation that you don't know how many rows you want but are sure of the number of columns
        return ids.view(batch_size, -1) 

In [24]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [25]:
# https://towardsdatascience.com/pytorch-basics-how-to-train-your-neural-net-intro-to-rnn-cb6ebc594677
# Hyper-parameters
embed_size = 128
hidden_size = 1024
num_layers = 2
num_epochs = 2
num_samples = 1000     # number of words to be sampled
batch_size = 30
seq_length = 15 #number of elements in each batch from the dimension
learning_rate = 0.002

#load the corpus (dictionary it is)
corpus = Corpus()
ids = corpus.get_data(news_train_clean, batch_size)
vocab_size = len(corpus.dictionary)
output_size = vocab_size
num_batches = ids.size(1) // seq_length
print(vocab_size)
val_ids = corpus.get_data(news_validation_clean,batch_size)
val_ids
#print word2idx
#print(corpus.dictionary.word2idx)

18226


tensor([[    0,    53,    71,  ...,    17,  4476,     5],
        [ 4056, 18246,     5,  ..., 18283,    14,     0],
        [  518,     7,    22,  ...,   125,   749,  3740],
        ...,
        [  130,  5385,  3275,  ...,  7020,     7,   226],
        [  142,    35,  4341,  ...,    86,   164,    28],
        [ 3567,     3, 18994,  ...,     1,  2695,  1681]])

In [26]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, output_size, embed_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, x, hidden):
        # Embed word ids to vectors
        x = self.embed(x)
        
        # Forward propagate LSTM
        out, (h, c) = self.lstm(x, hidden)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        # out = self.softmax(out)
        return out, (h, c)
    def init_hidden(self, batch_size):
        # Create two new tensors with sizes n_layers x batch_size x hidden_size,
        # initialized to zero, for hidden state and cell state of LSTM
        return (torch.zeros(num_layers, batch_size, hidden_size).to(device),
                torch.zeros(num_layers, batch_size, hidden_size).to(device))
model = LSTMModel(vocab_size, output_size, embed_size, hidden_size, num_layers)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
def detach (states):
    return [state.detach() for state in states]

In [27]:
num_epochs = 3
for epoch in range(num_epochs):
    # Set initial hidden and cell states 
    hidden = model.init_hidden(batch_size)
    states = hidden
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        # print(targets.shape)
        # print(outputs.shape)
        # print(outputs)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))



Epoch [1/3], Step[0/1028], Loss: 9.8033, Perplexity: 18093.75
Epoch [1/3], Step[100/1028], Loss: 6.0498, Perplexity: 424.03
Epoch [1/3], Step[200/1028], Loss: 6.0108, Perplexity: 407.80
Epoch [1/3], Step[300/1028], Loss: 5.6662, Perplexity: 288.95
Epoch [1/3], Step[400/1028], Loss: 5.6959, Perplexity: 297.65
Epoch [1/3], Step[500/1028], Loss: 5.3011, Perplexity: 200.56
Epoch [1/3], Step[600/1028], Loss: 5.5745, Perplexity: 263.61
Epoch [1/3], Step[700/1028], Loss: 5.9736, Perplexity: 392.92
Epoch [1/3], Step[800/1028], Loss: 5.4877, Perplexity: 241.71
Epoch [1/3], Step[900/1028], Loss: 5.4001, Perplexity: 221.44
Epoch [1/3], Step[1000/1028], Loss: 5.4159, Perplexity: 224.95
Epoch [2/3], Step[0/1028], Loss: 7.2309, Perplexity: 1381.46
Epoch [2/3], Step[100/1028], Loss: 5.1877, Perplexity: 179.06
Epoch [2/3], Step[200/1028], Loss: 5.1752, Perplexity: 176.83
Epoch [2/3], Step[300/1028], Loss: 5.0319, Perplexity: 153.23
Epoch [2/3], Step[400/1028], Loss: 5.1323, Perplexity: 169.41
Epoch [2

In [28]:
#saving best model checkpoint
torch.save(model.state_dict(), 'fr_model.ckpt')

In [15]:

#calculate perplexity of the input sentence using a model
import torch.nn.functional as F

def evaluate(model, sentence, vocab_size, device):
    model.eval()
    with torch.no_grad():
        sentence = preprocess(sentence)
        sentence = sentence.split()

        #replace words in sentence with UNK
        sentence = [word if word in corpus.dictionary.word2idx else '<UNK>' for word in sentence]
        #convert the sentence to ids
        ids = torch.tensor([[corpus.dictionary.word2idx[word] for word in sentence]])
        ids = ids.to(device)
        # Set initial hidden and cell states
        states = (torch.zeros(num_layers, 1, hidden_size).to(device),
                    torch.zeros(num_layers, 1, hidden_size).to(device))
        # Forward pass
        for i in range(len(sentence)):
            states = detach(states)
            _, states = model(ids[:, i:i+1], states)
        # Decode hidden states of all time steps
        out = states[0][0].squeeze(0)
        # for i in range(len(sentence)):
        #     print('{}: {}'.format(sentence[i], F.softmax(out[i], dim=0)))
    
        #print probability of each word in the sentence
        # for i in range(len(sentence)):
        #     print('{}: {}'.format(sentence[i], out[i].item()))
        #predict
        #print perplexity score of the sentence
        score = torch.exp(torch.log(torch.sum(torch.exp(out)))/len(sentence))
        return score.item()
  
evaluate(model, 'Les journalistes, personnalités à forte capacité médiatique et politique, rapportent le plus.', vocab_size, device)


1.3778657913208008